In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

In [3]:
params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}

In [4]:
ipdata = pd.read_excel('代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
# randomproxy = random.choice(proxieslist)

In [5]:
def task_monitor(task_list, mt_pool, results):
    count = 0
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result, key_word = future.result()
                print("当前关键词:",key_word) 
                print(len(result))
                count += 1
                results.extend(result)
                if count % 10 == 0:
                    with open('百度知道_药品问答/检索词.json', 'w', encoding='utf-8') as f:
                        f.write(json.dumps(results, indent=2, ensure_ascii=False)) 
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [6]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://zhidao.baidu.com', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [7]:
def get_qa_with_img(key_word):
    results = []
    page = 1
    e_count = 0
    url_num = 0
    headers_pc, randomproxy, cookie_dict = changeparams()
    while True:
        try:
            print(page)
            
            requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
            s = requests.session()
            s.keep_alive = False # 关闭多余连接
            
           #  changeparams()
            pn = (page-1)*10
            url = f'https://zhidao.baidu.com/search?pn={pn}&word="{key_word}"'
            r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
            r.encoding = 'utf-8'
            html = r.text

            selector = Selector(text=html)
            qa_list = selector.xpath("//div[@class='list-inner']/div/dl")
            # print(type(qa_list))
            for qa in qa_list:
                try:
                    url = qa.xpath('./dt/a/@href').extract()[0]
                    if "zhidao.baidu.com" not in url:
                        continue
                    if url is None or len(url) == 0:
                        continue
                    title = qa.xpath('./dt/a//text()').extract()
                    title = ''.join(title)
#                     print(title)
#                     print(url)
                    tmp = {
                        'title': title,
                        'url': url
                    }
                    url_num +=1
                    results.append(tmp)
#                     time.sleep(0.5)
                except Exception as e:
                    print(e)
                    pass
            
            
            page += 1
            
            if url_num > 200 or page > 30:
                break
                
            time.sleep(2)
            
            if page % 7 == 0:
                headers_pc, randomproxy, cookie_dict = changeparams()
                time.sleep(1)
   #             print(title)
#                print(url)
            
        except Exception as e:
            # randomproxy = random.choice(proxieslist)
            e_count += 1
            print(e)
            if e_count > 3:
                break
    return results, key_word

In [ ]:

if __name__ == "__main__":
    res = pd.read_excel('C:/Users/myzhang43/Desktop/百度知道_药品问答/药盒爬取-药品列表说明.xlsx', sheet_name='输入')
    keywords = res['药品名'][700:3064]
    
    all_craw_tasks = []
    results = []
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for keyword in keywords:
            tmp_task = mt_pool.submit(get_qa_with_img, keyword)
            all_craw_tasks.append(tmp_task)

        # 监测下载任务进展
        task_monitor(all_craw_tasks, mt_pool, results)
        
        with open('百度知道_药品问答/检索词7.24.json', 'w', encoding='utf-8') as f:
            f.write(json.dumps(results, indent=2, ensure_ascii=False))


1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
22

2
3
2
2
3
3
3
43

3
3
3
3
3
4
4
4
5
4
4
4
4
5
4
4
5
5
6
5
5
5
6
5
5
6
5
6
6
66

6
6
6
7
7
7
87

7
8
8
7
7
7
7
9
7
8
8
9
9
8
8
10
810

8
8
9
9
10
9
11
9
11
9
9
9
11
12
10
10
10
12
10
13
10
10
12
10
11
11
13
11
11
11
13
11
11
12
12
12
14
12
12
12
12
14
13
15
13
13
13
1314

16
15
13
13
17
15
16
14
14
14
18
14
16
17
14
15
19
15
15
1715

14
1814

15
20
16
16
16
18
15
16
19
15
list index out of range
16
17
19
17
17
20
1617

16
21
20
18
17
18
18
22
18
17
17
18
19
19
19
23
21
21
19
18
19
20
18
24
20
22
20
22
20
当前关键词: 美洛昔康片
205
25
19
当前关键词: 左氧氟沙星滴眼液
206
20
19
1
26
1
21
21
21
20
20
21
2
27
当前关键词: 大活络丸
208
当前关键词: 连翘败毒丸
206
2
21
22
1
3
1
28
3
2223

21
21
24

2
29
当前关键词: 芪苈强心胶囊
210
list index out of range
23
24
4
22
3
30
5
当前关键词: 裸花紫珠片
206
3
1
当前关键词: 注射用头孢西丁钠
126
22
25
1
list index out of range
45

4
2
23
6
1
26
5
5
当前关键词: 泮托拉唑钠肠溶胶囊
204
2
3
1
6
272

6
6
23
3
4
7
2
3
4
5
7
24
28
7
4
3
8
7
5
6
8
list index out of range
5
29
48

89

6
6
30
9
9
5
10
9
7
当前关键词: 维生素D3注射

5
16
HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=80&word=%22%E6%9E%B8%E6%A9%BC%E9%85%B8%E8%8E%AB%E6%B2%99%E5%BF%85%E5%88%A9%E5%88%86%E6%95%A3%E7%89%87%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002E87E0CBA60>, 'Connection to zhidao.baidu.com timed out. (connect timeout=10)'))
9
17
2
12
HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=90&word=%22%E5%BD%93%E5%BD%92%E8%8B%A6%E5%8F%82%E4%B8%B8%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002E87E0CBA30>, 'Connection to zhidao.baidu.com timed out. (connect timeout=10)'))
10
HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=10&word=%22%E5%A4%A7%E5%B1%B1%E6%A5%82%E4%B8%B8%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002

20
HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=40&word=%22%E5%8F%8C%E5%94%91%E6%B3%B0%E8%BD%AF%E8%86%8F%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002E87CEF5CA0>, 'Connection to zhidao.baidu.com timed out. (connect timeout=10)'))
5
HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=160&word=%22%E5%BD%93%E5%BD%92%E8%8B%A6%E5%8F%82%E4%B8%B8%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002E87E0CB760>, 'Connection to zhidao.baidu.com timed out. (connect timeout=10)'))HTTPSConnectionPool(host='zhidao.baidu.com', port=443): Max retries exceeded with url: /search?pn=110&word=%22%E6%B0%AF%E9%9B%B7%E4%BB%96%E5%AE%9A%E8%83%B6%E5%9B%8A%22&ie=UTF-8&wd=Python (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002E87B701AC0>, 'Connection to zhidao.baid

12
9
1
22
17
11
13
1
19
10
1723

18
21
12
20
211

14
24
2
19
13
22
12
25
3
15
2018

23
26
21
13
16
3
4
24
27
list index out of range
17
14
22
5
4
18
21
当前关键词: 曲安奈德益康唑乳膏
209
25
14
15
28
6
5
当前关键词: 呋塞米注射液
207
19
15
1
29
26
1
16
6
20
19
16
30
当前关键词: 复方嗜酸乳杆菌片
201
7
17
2
当前关键词: 盐酸金霉素软膏
167
2
171

21
8
18
1
7
18
当前关键词: 安胃疡胶囊
201
3
3
2
1
list index out of range
9
19
8
419

20
4
2
2
3
2010

920

55

3
3
4
11
6
410

21
6
4
21
21
5
22当前关键词: 盐酸曲美他嗪片
203

11
125

1
22
5
7
7
当前关键词: 来氟米特片
209
23
6
6
131

12
2
88

24
6
3
2
13
7
7
25
14
9
9
4
8
3
268

15
10
10
7
5
27
14
9
4
9
16
11
11
8
156

5
10
10
28
17
list index out of range
12
11
9
6
11
12
16
29
18
12
7
13
1013

30
17
12
当前关键词: 宫炎康颗粒
133
19
list index out of range
13
7
11
8
18
1
13
20
list index out of range8

12
14
14
9
2
14
19
9
13
15
15
10
15
21
3
14
10
20
16
16
1611

14
11
22
15
4
17
17
17
1215

1221

16
23
5
18
18
13
2218
16

17
24
6
13
19
19
23
17list index out of range

19
25
20
18
list index out of range
20
141824


20
7
26
14
19
19
25
15

26
20
27
8
7
list index out of range
22
16
21
19
27
11
8
list index out of rangelist index out of range
9

23
28
21
17
12
20
22
当前关键词: 卤米松乳膏
207
list index out of range
9
10
当前关键词: 海狗丸
210
29
1
13
23
28
1
18
10
11
30
当前关键词: 百癣夏塔热片
209
21
2
29
192

list index out of range12
当前关键词: 复方草珊瑚含片
202

11
1
22
3
14
30
list index out of range
120

12
3
当前关键词: 黄藤素片
194
23
13
2
4
15
list index out of range
1
list index out of range
13
4
24
2
5
list index out of range
16
2
3
2114

25
5
63

list index out of range
17
3
1426

15
4
4
22
6
list index out of range
4
18
27
15
16
5
7
5
23
5
list index out of range
19
16
68

6
17
6
7
24
28
20
17
9
list index out of range
18
8
29
25
7
18
10
7
list index out of range
7
30
19
89

21
19
26
11
list index out of range
当前关键词: 克痒舒洗液
155
8
9
20
822

list index out of range
10
1
12
20
10
9
27
23
9
2
11
11
13
21
10
2412

3
2112

10
22
13
25
4
11
22
28
14
13
23
11
当前关键词: 人胎盘片
201
5
12
15
23
1
29
24
14
12
6
13
16
24
15
30
2
14
25
13
17
当前关键词: 盐酸丙卡特罗口服溶液
69
16
25
3
15
26

...

14
212

23
4
29
1
21
5
22
15
2
24
3
5
30
2
16
22
23
当前关键词: 氟米龙滴眼液
196
2543

6

6
34

24
17
1
23
5
26
25
当前关键词: 盐酸氨基葡萄糖片
209
5
18
42

7
6
27
1
26
7
3
6
19
8
5
2
27
4
8
20
28
7
6
9
3
7
5
28
9
829

8
10
6
21
4
3029

7
10
11
9
当前关键词: 复方氨肽素片
154
9
221

30
5
8
12
当前关键词: 氧氟沙星栓
206
10
7
11
1
23
10
6
2
9
811

13
12
2
24
11
9
3
10
12
13
25
3
12
10
7
14
4
11
13
26
11
8
15
13
4
14
5
12
12
9
27
16
5
156

14
13
13
14
10
6
17
16
15
28
15
11
7
7
29
14
17
14
18
16
16
81518

30
12

15
8
19
17
17
当前关键词: 镇脑宁胶囊
117
9
16
13
9
1
19
20
16
18
list index out of range
1817

10
10
17
2
20
19
18
19
18
14
2111

20113


19
20
22
21
19
15
4
12
20
12
22
23
21
521

20
16
23
24
13
13
22
22
21
617

25
24
2322

2618

21
23
14
25
14
23
24
7
19
22
27
24
26
15
15
24
25
8
20
25
23
25
27
16
28
16
9
26
26
26
24
29
21
17
17
27
2728

10
2730

22
25
18
当前关键词: 羚羊清肺颗粒
143
29
18
当前关键词: 安乃近片
210
11
1
1
26
28
1930

19
28
当前关键词: 利巴韦林泡腾颗粒
86
12
28
2
2
27
1
29
20
20
29
29
13
3
3
30
2
30
当前关键词: 复方鱼腥草颗粒
159
30
4
4
28
21
当前关键词: 1
风湿液
187
21


6
6
list index out of range14

8
16
12
15
14
7
14
list index out of range
17
15
169

13
715

8
15
718

17
1610

8
list index out of range
16
916

14
8
19
17
18
11
9
17
17
20
10
15
10199


18
12
list index out of range
list index out of range
list index out of range
18
18
11
20
19
11
16
13
10
21
19
19
list index out of range
17
12
2011

12
22
20
20
21
list index out of range
list index out of range
list index out of range
14
13
18
12
13
23
22
当前关键词: 茶苯海明片
204
21
1319

15
21
21
23
22
1
22
16
1414

20
22
24
15
23
14
15
23
2当前关键词:
 普适泰片
204
17
23
24
24
16
16
21
1
list index out of range
15
3
18
25
24
25
17
22
17list index out of range

list index out of range
16
2
4
19
当前关键词: 乳安片
206
26
25
23
18
1
18
17
3
5
list index out of range
当前关键词: 尿素乳膏
206
20
27
26
19
2
19
18
1
4
6
3
20
27
20
5
21
28
19
2
当前关键词: 多糖铁复合物胶囊
210
4
6
22
1
29
21
7
21
5
20
3
list index out of range
2
23
8
30
6
4
22
22
当前关键词: 清热暗疮丸
201
当前关键词: 头孢羟氨苄颗粒
144
3
7
5
9
23
21
23
4
11

7
6
8
24
当前关键词: 转移因子口服溶液
209
10
22
list index o

19
15
10
28
17
7
13
29
3
2029

16
11
30
81729


4
18
当前关键词:30
 益母草颗粒(无糖型)
0
当前关键词: 呋喃妥因片
97
21
1
12
18
1
9
30
5
14
19
22
当前关键词: 复方硫酸亚铁颗粒
207
2
19
1013

6
1
2
15
23
20
20
11
3
243

16
2
2512

4
14
7
17
4
21
3
26
5
21
13
8
15
5
18
22
27
4
22
6
9
166
1923


当前关键词: 盐酸特拉唑嗪片
207
1
5
14
10
28
20
24
17
15
29
11
25
6
7
7
18
2
30
26
16
12
21
当前关键词: 阿仑膦酸钠片
169
8
198

27
17
1
3
13
7
22
9
20
9
list index out of range
list index out of range
18
4
2
23
10
8
28
10
19
14
5
3
11
24
29
21
9
20
11
15
12
6
30
25
4
22
10
当前关键词: 司帕沙星分散片
193
1612

13
23
5
1
21
26
11
17
13
7
24
22
27
6
18
14
2
12
23
25
8
14
19
15
当前关键词: 众生丸
210
3
13
28
1
26
20
7
9
16
15
4
271029


8
17
2
5
16
21
14
6
9
30
18
11
17
3
22
15
list index out of range
19当前关键词: 龙胆泻肝胶囊
125

10
28
18
12
4
23
1
当前关键词: 保妇康泡沫剂
203
20
16
7
11
24
19
513

1
2
25
17
12
8
21
6
20
2
9
26
14
22
3
13
3
18
list index out of range
107

21
27
4
15
23
4
19
14
11list index out of range

22
8
5
16
245

20
15
2312

6
28
9
当前关键词: 人参健脾片
205
25
当前关键词: 肥儿丸
202
1
13
17
6
1
1

18
9
22
21
23
10
7
26
19
15
当前关键词: 复方益肝灵片
205
10
11
24
8
16
20
7
27
11
1
12
25
list index out of range
list index out of range
917

8
12
13
26
2
18
10
13
9
28
21
27
22
19
list index out of range
3
11
10
14
22
29
14
28
list index out of range当前关键词:
 布拉氏酵母菌散
207
list index out of range
23
list index out of range
list index out of range
list index out of range
4
20
1211

1
15
23
15
29
当前关键词: 夏枯草口服液
203
125

13
1
16
16
230

当前关键词: 香菇菌多糖片
152
21
24
13
17
6
17
1
32

14
22
1818

25
4
23

14
723

1915

19
5
4
15
26
3
8
24
20
20
16
6
5
16
27
4
9
25
17
21
610

26
5
17
7
18
22
21
27
6
11
28
23
822

19
7
12
24
23
299

20
8
25
13
28
7
24
10
30
26
25
9
29
8
当前关键词: 化积口服液
165
11
当前关键词: 金天格胶囊
202
21
27
18
1
1
14
30
10
9
22
12
当前关键词: 牡蛎碳酸钙泡腾片
190
2
15
1011

28
2
23
1
13
3
16
29
12
11
24
2
3
4
17
30
13
12
当前关键词: 复方鸡内金片
141
25
5
18
3
4
14
1
13
4
19
6
5
14
26
15
2
20
5
6
15
19
27
3
16
6
14
7
16
4
1517

21
8
list index out of range7

517

28
16
18
22
7
9
6
8
18
list index out of range
29
23
8
10
17
19
924
2

3
29
25
2
16
13
10
29
当前关键词: 柴胡注射液
206
30
4
当前关键词: 盐酸美他环素胶囊
177
17
3
1
30
14
28
11
当前关键词: 复方青黛丸
157
1
1
14
2
15
29
18
5
4
12
2
16
15
30
2
3
19
5
6
13
当前关键词: 盐酸异丙嗪注射液
188
3
17
16
4
1
3
20
6
18
5
17
4
2
4
19
6
7
14
18
5
3
20
5
21
19
8
15
6
4
7
7
6
22
20
5
16
8
9
21
8
23
6
9
22
17
7
10
9
21
107

24
23
18
8
11
11
22
108

7
25
24
9
19
12
23
8
12
9
25
26
1013

11
20
24
9
26
13
10
当前关键词: 还少胶囊
209
27
11
10
12
27
11
1
11
12
1421

13
14
12
12
15
2
13
28
28
22
16
13
15
3
13
29
29
14
17
23
16
30
4
14
18
30
15
14
24
当前关键词: 三宝胶囊
193
当前关键词: 宫瘤消胶囊
189
19
5
1
15
15
17
25
1
14
20
16
616

18
16
2
26
15
17
17
21
17
2
7
19
3
18
27
18
22
16
8
19
18
23
320

4
19
17
9
20
24
list index out of range
19
28
5
4
1820

10
25
20
21
1129

21
19
6
5
26
12
22
22
30
21
27
20
当前关键词: 盐酸依匹斯汀胶囊
191
6
13
21
23
23
1
22
7
24list index out of range

22
24
21
28
当前关键词: 雷公藤片
207
14
25
8
2
23
7
29
1
22
25
1526

30
249

3
8
27
当前关键词: 健胃消食颗粒
32
2
23
当前关键词: 丁酸氢化可的松乳膏
207
16
25
4
10
1
1
17
9
24
3
当前关键词: 丙酸倍氯米松气雾剂
204
28
11
18
1
5
25
2